In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# data_low = pd.read_csv('../datasets/data_value_low.csv', dtype={'거래소코드':'object'}, index_col=0)
data_low = pd.read_csv('datasets/data_value_low.csv', dtype={'거래소코드':'object'}, index_col=0)
data_low.shape

(11021, 12)

In [3]:
data_low.head()

,year,회사명,거래소코드,회계년도,PER(최고),PER(최저),PBR(최고),PBR(최저),PCR(최고),PCR(최저),PSR(최고),PSR(최저)
5384,2008,삼성전자(주),005930,2007/12,15.74,11.46,2.34,1.70,8.81,6.41,1.85,1.35
5385,2008,포스코홀딩스(주),005490,2007/12,18.13,6.74,2.81,1.05,13.15,4.89,3.00,1.12
5386,2008,한국전력공사(주),015760,2007/12,19.86,15.08,0.71,0.54,12.13,9.21,1.07,0.81
5387,2008,에스케이텔레콤(주),017670,2007/12,13.77,9.27,2.45,1.65,6.08,4.09,2.00,1.35
5388,2008,에이치디한국조선해양(주),009540,2007/12,24.08,5.25,8.73,1.90,10.16,2.22,2.69,0.59


In [4]:
data_low.tail()

,year,회사명,거래소코드,회계년도,PER(최고),PER(최저),PBR(최고),PBR(최저),PCR(최고),PCR(최저),PSR(최고),PSR(최저)
17333,2023,제이준코스메틱(주),025620,2022/12,0.0,0.0,13.29,0.00,0.0,0.0,99.19,0.00
17334,2023,(주)대유플러스,000300,2022/12,0.0,0.0,3.30,1.65,0.0,0.0,1.73,0.86
17335,2023,비케이탑스(주),030790,2022/12,0.0,0.0,11.50,0.00,0.0,0.0,21.48,0.00
17336,2023,(주)코아스,071950,2022/12,0.0,0.0,3.01,1.43,0.0,0.0,0.49,0.23
17337,2023,일정실업(주),008500,2022/12,0.0,0.0,6.11,2.19,0.0,0.0,1.70,0.61


In [5]:
# 결측치 없음

data_low.isna().sum()

year       0
회사명        0
거래소코드      0
회계년도       0
PER(최고)    0
PER(최저)    0
PBR(최고)    0
PBR(최저)    0
PCR(최고)    0
PCR(최저)    0
PSR(최고)    0
PSR(최저)    0
dtype: int64

### (가치주) - 저PER, ...

- 데이터프레임명 : `data_low`

In [6]:
data_low

,year,회사명,거래소코드,회계년도,PER(최고),PER(최저),PBR(최고),PBR(최저),PCR(최고),PCR(최저),PSR(최고),PSR(최저)
5384,2008,삼성전자(주),005930,2007/12,15.74,11.46,2.34,1.70,8.81,6.41,1.85,1.35
5385,2008,포스코홀딩스(주),005490,2007/12,18.13,6.74,2.81,1.05,13.15,4.89,3.00,1.12
5386,2008,한국전력공사(주),015760,2007/12,19.86,15.08,0.71,0.54,12.13,9.21,1.07,0.81
5387,2008,에스케이텔레콤(주),017670,2007/12,13.77,9.27,2.45,1.65,6.08,4.09,2.00,1.35
5388,2008,에이치디한국조선해양(주),009540,2007/12,24.08,5.25,8.73,1.90,10.16,2.22,2.69,0.59
...,...,...,...,...,...,...,...,...,...,...,...,...
17333,2023,제이준코스메틱(주),025620,2022/12,0.00,0.00,13.29,0.00,0.00,0.00,99.19,0.00
17334,2023,(주)대유플러스,000300,2022/12,0.00,0.00,3.30,1.65,0.00,0.00,1.73,0.86
17335,2023,비케이탑스(주),030790,2022/12,0.00,0.00,11.50,0.00,0.00,0.00,21.48,0.00
17336,2023,(주)코아스,071950,2022/12,0.00,0.00,3.01,1.43,0.00,0.00,0.49,0.23
